In [22]:
import pandas as pd
import os, re, sqlite3
import numpy as np
import gc # 메모리 관리를 위한 가비지 컬렉터

# === Cell 0: Paths & Imports (경로 설정) ===
# ⚠️ 현재 MyDrive 바로 아래 파일 위치에 맞춰 수정된 경로입니다.
DB_PATH = "/content/drive/MyDrive/MIMIC4-hosp-icu.db"
COHORT_CSV = "/content/drive/MyDrive/cohort.csv"
SAVE_PATH  = "/content/drive/MyDrive/final_cohort_with_timevars.csv"

conn_settings = {
    'journal_mode': 'OFF',
    'synchronous': 'OFF',
    'temp_store': 'MEMORY',
    'cache_size': -500000
}
def get_db_connection(path):
    conn = sqlite3.connect(f"file:{path}?mode=ro", uri=True)
    for key, value in conn_settings.items():
        conn.execute(f"PRAGMA {key}={value};")
    conn.commit()
    return conn

# DB 연결 (Cell 0에서 DB_PATH만 사용하여 연결을 시도하는 경우를 대비)
try:
    conn = get_db_connection(DB_PATH)
    conn.close()
    print("DB connection test successful.")
except Exception as e:
    print(f"DB 연결 실패: {e}")

# === Cell 1: Load cohort.csv & Fix Timezone Error ===
print("\n=== Cell 1: Load cohort.csv 시작 ===")
try:
    cohort = pd.read_csv(COHORT_CSV)
    print(f"[선행 1] cohort.csv 로드 완료 (Shape: {cohort.shape})")

    # 시간 컬럼 자동 파싱 및 UTC 통일 (Timezone Error 해결)
    for c in cohort.columns:
        if any(k in c.lower() for k in ["time","date","intime","outtime","admit","disch","ecg"]):
            cohort[c] = pd.to_datetime(cohort[c], errors="coerce", utc=True)

    # door_to_ecg_min 컬럼 안정화: 시간차가 이미 계산된 '분' 단위이므로 float으로 변환
    if 'door_to_ecg_min' in cohort.columns:
        cohort['door_to_ecg_min'] = pd.to_numeric(cohort['door_to_ecg_min'], errors='coerce')
        if np.issubdtype(cohort['door_to_ecg_min'].dtype, np.datetime64):
             cohort['door_to_ecg_min'] = (cohort['door_to_ecg_min'] - cohort['door_to_ecg_min'].min()).dt.total_seconds() / 60.0

    print(f"✅ Timezone 및 door_to_ecg_min 안정화 완료.")

except Exception as e:
    print(f"❌ ERROR in Cell 1: {e}")


# === Cell 2: admissions merge & Boarding Delay ===
print("\n=== Cell 2: admissions merge & Boarding Delay 시작 ===")
try:
    conn = get_db_connection(DB_PATH)

    admissions = pd.read_sql("""
        SELECT subject_id, hadm_id, admittime, dischtime, admission_location
        FROM admissions
    """, conn, parse_dates=["admittime","dischtime"])

    conn.close()

    # 시간대 UTC 통일 및 ID 다운캐스팅
    admissions["admittime"] = admissions["admittime"].dt.tz_localize('UTC')
    admissions["dischtime"] = admissions["dischtime"].dt.tz_localize('UTC')
    admissions['subject_id'] = admissions['subject_id'].astype('int32')
    admissions['hadm_id'] = admissions['hadm_id'].astype('Int32')

    cohort2 = cohort.merge(
        admissions, on=["subject_id","hadm_id"], how="left"
    )

    # Boarding Delay = (admittime - outtime) [분]
    cohort2["boarding_delay_min"] = (cohort2["admittime"] - cohort2["outtime"]).dt.total_seconds() / 60.0

    print("After admissions merge:", cohort2.shape)

    del admissions
    del cohort
    gc.collect()

except Exception as e:
    print(f"❌ ERROR in Cell 2: {e}")


# === Cell 3: Lab TAT (채혈→결과기록 시간) 계산 ===
# ⚠️ 이 코드는 Lab TAT 계산 후 cohort2에 병합하는 로직을 포함합니다.
print("\n=== Cell 3: Lab TAT 계산 시작 ===")
try:
    conn = get_db_connection(DB_PATH)

    d_labitems = pd.read_sql("SELECT itemid, label FROM d_labitems", conn)
    lab_targets = d_labitems[d_labitems["label"].str.contains(r"(?:troponin|ck[-\s]?mb)", flags=re.I, regex=True, na=False)].copy()
    target_itemids = sorted(set(lab_targets["itemid"].tolist()))

    tmp = cohort2.dropna(subset=["intime","admittime"])[["subject_id","hadm_id","intime","admittime"]].drop_duplicates().copy()
    tmp["win_start"] = tmp["intime"]
    tmp["win_end"]   = tmp["admittime"] + pd.Timedelta(hours=24)

    hadms = tmp["hadm_id"].dropna().unique().tolist()
    BATCH = 1500
    all_parts = []

    for i in range(0, len(hadms), BATCH):
        batch = hadms[i:i+BATCH]
        placeholders = ",".join(["?"]*len(batch))
        sql = f"""
            SELECT subject_id, hadm_id, charttime, storetime, itemid
            FROM labevents
            WHERE itemid IN ({",".join(map(str, target_itemids))})
              AND hadm_id IN ({placeholders})
        """
        le = pd.read_sql(sql, conn, params=batch, parse_dates=["charttime","storetime"])

        le["charttime"] = le["charttime"].dt.tz_localize('UTC')
        le["storetime"] = le["storetime"].dt.tz_localize('UTC')

        if le.empty: continue

        merged = le.merge(tmp[["subject_id","hadm_id","win_start","win_end"]],
                          on=["subject_id","hadm_id"], how="inner")

        in_win = merged[
            (merged["charttime"].between(merged["win_start"], merged["win_end"])) |
            (merged["storetime"].between(merged["win_start"], merged["win_end"]))
        ].copy()

        if not in_win.empty:
            in_win["lab_tat_min"] = (in_win["storetime"] - in_win["charttime"]).dt.total_seconds()/60.0
            all_parts.append(in_win[["subject_id","hadm_id","lab_tat_min"]])

    conn.close()

    if all_parts:
        lab_rows = pd.concat(all_parts, ignore_index=True)
        lab_tat  = lab_rows.groupby(["subject_id","hadm_id"], as_index=False)["lab_tat_min"].median()
    else:
        lab_tat  = pd.DataFrame(columns=["subject_id","hadm_id","lab_tat_min"])

    print("Lab TAT available hadm:", len(lab_tat))

    cohort2 = cohort2.merge(lab_tat, on=["subject_id","hadm_id"], how="left")

except Exception as e:
    print(f"❌ ERROR in Cell 3: {e}")
finally:
    gc.collect()

# === Cell 4: Door-to-Antithrombotic (IV) 계산 ===
# ⚠️ 이 코드는 Antithrombotic 계산 후 cohort2에 병합하는 로직을 포함합니다.
print("\n=== Cell 4: Door-to-Antithrombotic (IV) 계산 시작 ===")
try:
    conn = get_db_connection(DB_PATH)

    hadms = cohort2["hadm_id"].dropna().unique().tolist()
    BATCH = 2000
    parts = []

    drug_keywords = [r"\bheparin\b", r"\bbivalirudin\b", r"\beptifibatide\b",
                     r"\babciximab\b", r"\btirofiban\b", r"\bclopidogrel\b", r"\baspirin\b"]
    pat = re.compile("|".join(drug_keywords), flags=re.I)

    for i in range(0, len(hadms), BATCH):
        batch = hadms[i:i+BATCH]
        placeholders = ",".join(["?"]*len(batch))
        sql = f"""
            SELECT subject_id, hadm_id, starttime, drug, route
            FROM prescriptions
            WHERE hadm_id IN ({placeholders})
              AND starttime IS NOT NULL
        """
        df = pd.read_sql(sql, conn, params=batch, parse_dates=["starttime"])

        df["starttime"] = df["starttime"].dt.tz_localize('UTC')

        if df.empty: continue

        df["drug_lc"] = df["drug"].astype(str).str.lower()
        df["route_lc"] = df["route"].astype(str).str.lower()

        # IV + 항혈전/항혈소판 후보
        cand = df[
            df["drug_lc"].str.contains(pat, na=False) &
            df["route_lc"].str.contains(r"\biv\b|intravenous", na=False)
        ].copy()

        if cand.empty: continue

        cand = cand.merge(cohort2[["subject_id","hadm_id","intime"]],
                          on=["subject_id","hadm_id"], how="inner")
        cand["door_to_antithrombotic_min"] = (cand["starttime"] - cand["intime"]).dt.total_seconds()/60.0

        g = cand.groupby(["subject_id","hadm_id"], as_index=False)["door_to_antithrombotic_min"].min()
        parts.append(g)

    conn.close()

    if parts:
        anti_time = pd.concat(parts, ignore_index=True)
    else:
        anti_time = pd.DataFrame(columns=["subject_id","hadm_id","door_to_antithrombotic_min"])

    print("Antithrombotic (IV) available hadm:", len(anti_time))

    cohort2 = cohort2.merge(anti_time, on=["subject_id","hadm_id"], how="left")

except Exception as e:
    print(f"❌ ERROR in Cell 4: {e}")
finally:
    gc.collect()

# === Cell 5: Door-to-Cath (시술 시각) 계산 - 민감도 강화 및 누락 셀 복구 ===
# ⚠️ 이 코드는 n=75 문제를 해결하기 위한 '민감도 강화' 로직입니다.
print("\n=== Cell 5: Door-to-Cath 계산 시작 (n=75 개선 목표) ===")

try:
    conn = get_db_connection(DB_PATH)

    # ---- 옵션 플래그 ----
    USE_PROC_ICD_PREFILTER = True
    INCLUDE_CABG_AS_CATH   = False

    # ---- 정규식(라벨/유닛) 보강 ----
    label_patterns = [r"\bcath\b|cath\s*lab", r"catheter(?:ization)?", r"coronary", r"angiograph(?:y|ie)?", r"angioplast(?:y|ie)?", r"\bpci\b", r"\bstent\b", r"interventional cardiology", r"left heart cath", r"right heart cath", r"\bPTCA\b", r"\bPTCA-A\b", r"balloon", r"dilation", r"angio", r"stent placement", r"percutaneous intervention"]
    kw_label = re.compile("|".join(label_patterns), flags=re.I)

    unit_patterns = [r"\bcath\b|cath\s*lab", r"cardiac cath", r"cardiac catheteriz", r"interventional|cardio interventional", r"\bccl\\b", r"ep/cath", r"coronary angiography", r"ICU.*CATH", r"cath.*ICU"]
    kw_unit = re.compile("|".join(unit_patterns), flags=re.I)

    # ---- 0) procedures_icd로 cath-가능 hadm 먼저 좁히기 ----
    def get_likely_cath_hadm(conn, cohort_hadm_df):
        proc_icd = pd.read_sql("""
            SELECT subject_id, hadm_id, icd_version, icd_code
            FROM procedures_icd
            WHERE hadm_id IN (SELECT DISTINCT hadm_id FROM admissions)
        """, conn)

        proc_icd['subject_id'] = proc_icd['subject_id'].astype('int32')
        proc_icd['hadm_id'] = proc_icd['hadm_id'].astype('Int32')

        proc_icd["code_clean"] = (proc_icd["icd_code"].astype(str).str.replace(".","", regex=False).str.upper())

        # ICD-9: PCI/관상동맥 조영/확장 (연구 제안서 ICD 확장 적용)
        mask9 = (proc_icd["icd_version"]==9) & (
            proc_icd["code_clean"].str.startswith(("360","0066","8855")) |
            proc_icd["code_clean"].str.match(r"(0066|8855)", na=False)
        )
        # ICD-10-PCS: 조영(B211/B212), PCI(0270~0273*)
        mask10 = (proc_icd["icd_version"]==10) & (
            proc_icd["code_clean"].str.match(r"(B211|B212|0270|0271|0272|0273)", na=False)
        )

        likely = proc_icd[mask9 | mask10][["subject_id","hadm_id"]].drop_duplicates()
        likely = likely.merge(cohort_hadm_df[["subject_id","hadm_id"]].drop_duplicates(),
                              on=["subject_id","hadm_id"], how="inner")

        del proc_icd
        gc.collect()
        return likely

    if USE_PROC_ICD_PREFILTER:
        likely_cath = get_likely_cath_hadm(conn, cohort2)
        hadms = likely_cath["hadm_id"].unique().tolist()
        print("procedures_icd 기반 cath-가능 hadm:", len(hadms))
    else:
        hadms = cohort2["hadm_id"].dropna().unique().tolist()

    hadms.sort()

    # ---- d_items 라벨 준비 ----
    try:
        d_items = pd.read_sql("SELECT itemid, label FROM d_items", conn)
        d_items["label_lc"] = d_items["label"].astype(str).str.lower()
    except Exception:
        d_items = pd.DataFrame(columns=["itemid","label","label_lc"])

    # ---- A) procedureevents 기반 탐색 ----
    BATCH = 2000
    proc_parts = []

    for i in range(0, len(hadms), BATCH):
        batch = hadms[i:i+BATCH]
        placeholders = ",".join(["?"]*len(batch))
        sql = f"""
            SELECT subject_id, hadm_id, starttime, endtime, itemid
            FROM procedureevents
            WHERE hadm_id IN ({placeholders})
              AND starttime IS NOT NULL
        """
        pe = pd.read_sql(sql, conn, params=batch, parse_dates=["starttime","endtime"])
        if pe.empty: continue

        pe["starttime"] = pe["starttime"].dt.tz_localize('UTC')
        pe["endtime"] = pe["endtime"].dt.tz_localize('UTC')

        if not d_items.empty:
            pe = pe.merge(d_items[["itemid","label_lc"]], on="itemid", how="left")
        else:
            pe["label_lc"] = ""

        pe_cath = pe[ pe["label_lc"].str.contains(kw_label, na=False) ].copy()
        if pe_cath.empty: continue

        pe_cath = pe_cath.merge(cohort2[["subject_id","hadm_id","intime"]], on=["subject_id","hadm_id"], how="inner")
        pe_cath["door_to_cath_min_A"] = (pe_cath["starttime"] - pe_cath["intime"]).dt.total_seconds()/60.0
        gA = pe_cath.groupby(["subject_id","hadm_id"], as_index=False)["door_to_cath_min_A"].min()
        proc_parts.append(gA)
        # print(f"proc batch {i//BATCH+1}: rows={len(pe)}, cath={len(pe_cath)}, hadm_A={len(gA)}")

    cath_A = (pd.concat(proc_parts, ignore_index=True)
              if proc_parts else
              pd.DataFrame(columns=["subject_id","hadm_id","door_to_cath_min_A"]))

    # ---- B) transfers 기반 탐색 ----
    trf_parts = []
    for i in range(0, len(hadms), BATCH):
        batch = hadms[i:i+BATCH]
        placeholders = ",".join(["?"]*len(batch))
        sql = f"""
            SELECT subject_id, hadm_id, intime, outtime, careunit
            FROM transfers
            WHERE hadm_id IN ({placeholders})
              AND intime IS NOT NULL
        """
        tr = pd.read_sql(sql, conn, params=batch, parse_dates=["intime","outtime"])
        if tr.empty: continue

        tr["intime"] = tr["intime"].dt.tz_localize('UTC')
        tr["outtime"] = tr["outtime"].dt.tz_localize('UTC')

        tr["careunit_lc"] = tr["careunit"].astype(str).str.lower()
        tr_cath = tr[ tr["careunit_lc"].str.contains(kw_unit, na=False) ].copy()
        if tr_cath.empty: continue

        tr_cath = tr_cath.merge(cohort2[["subject_id","hadm_id","intime"]], on=["subject_id","hadm_id"], how="inner", suffixes=("","_ed"))
        tr_cath["door_to_cath_min_B"] = (tr_cath["intime"] - tr_cath["intime_ed"]).dt.total_seconds()/60.0
        gB = tr_cath.groupby(["subject_id","hadm_id"], as_index=False)["door_to_cath_min_B"].min()
        trf_parts.append(gB)
        # print(f"trf batch {i//BATCH+1}: rows={len(tr)}, cath_unit={len(tr_cath)}, hadm_B={len(gB)}")

    conn.close()

    cath_B = (pd.concat(trf_parts, ignore_index=True)
              if trf_parts else
              pd.DataFrame(columns=["subject_id","hadm_id","door_to_cath_min_B"]))

    # ---- A 우선, 없으면 B로 보완 ----
    door_to_cath = (cohort2[["subject_id","hadm_id"]]
                    .drop_duplicates()
                    .merge(cath_A, on=["subject_id","hadm_id"], how="left")
                    .merge(cath_B, on=["subject_id","hadm_id"], how="left"))

    door_to_cath["door_to_cath_min"] = door_to_cath["door_to_cath_min_A"].combine_first(
        door_to_cath["door_to_cath_min_B"]
    )

    # ⚠️ [변수 정의] 이 변수가 Cell 6에서 사용됩니다.
    cath_df = door_to_cath.copy()

    print("Door-to-Cath (A or B):", cath_df["door_to_cath_min"].notna().sum())

except Exception as e:
    print(f"❌ ERROR in Cell 5: {e}")
finally:
    gc.collect()

# === Cell 6: 시간변수 병합 및 최종 저장 ===
print("\n=== Cell 6: 시간변수 병합 및 최종 저장 시작 ===")
try:
    if 'cath_df' not in locals():
         cath_df = pd.DataFrame(columns=["subject_id", "hadm_id", "door_to_cath_min"])
         print("⚠️ cath_df 변수가 정의되지 않아 빈 DataFrame으로 병합을 시도합니다.")

    # 1. Door-to-Cath 병합 (A, B 방법 중 최소값)
    cohort2 = cohort2.merge(cath_df[["subject_id", "hadm_id", "door_to_cath_min"]],
                            on=["subject_id", "hadm_id"], how="left")

    # 2. Boarding Delay는 Cell 2에서 이미 계산됨

    # 3. 최종 DF 정리 및 인덱스 초기화
    out = cohort2.copy()
    out.reset_index(drop=True, inplace=True)

    print(f"Merged final shape: {out.shape}")

    # 4. 최종 저장 (CSV)
    out.to_csv(SAVE_PATH, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 코호트 파일 저장 완료")
    print(f"저장 경로: {SAVE_PATH}")

except Exception as e:
    print(f"❌ ERROR in Cell 6: {e}")
finally:
    gc.collect()


DB connection test successful.

=== Cell 1: Load cohort.csv 시작 ===
[선행 1] cohort.csv 로드 완료 (Shape: (465, 11))
✅ Timezone 및 door_to_ecg_min 안정화 완료.

=== Cell 2: admissions merge & Boarding Delay 시작 ===
After admissions merge: (465, 15)

=== Cell 3: Lab TAT 계산 시작 ===
Lab TAT available hadm: 433

=== Cell 4: Door-to-Antithrombotic (IV) 계산 시작 ===
Antithrombotic (IV) available hadm: 301

=== Cell 5: Door-to-Cath 계산 시작 (n=75 개선 목표) ===
procedures_icd 기반 cath-가능 hadm: 464
Door-to-Cath (A or B): 75

=== Cell 6: 시간변수 병합 및 최종 저장 시작 ===
Merged final shape: (465, 18)

✅ 최종 코호트 파일 저장 완료
저장 경로: /content/drive/MyDrive/final_cohort_with_timevars.csv


In [44]:
# === Cell 5 (v2.2): Door-to-Cath proxy with ICD prefilter + maximal regex + safety guards ===
import re, sqlite3
import pandas as pd
import gc
import numpy as np

# ---- 옵션 ----
USE_PROC_ICD_PREFILTER = True
INCLUDE_CABG_AS_CATH   = False       # 필요 시 CABG도 포함
ALLOW_NEG_MINUTES      = 30          # 문과 시계 불일치 허용 범위(분) : -30분까지 허용
MAX_MINUTES_WINDOW     = 72*60       # 72시간 초과 값은 이상치로 제거

# ---- 정규식 ----
# procedureevents.label 후보 (A)
label_patterns = [
    r"heart\s*cath", r"cardiac\s*proc", r"cardiac\s*interven",
    r"\bcath\b", r"catheter(?:ization)?",
    r"cor\s*angio(?:gram|graphy|grafi|gra|)?",
    r"coronary",
    r"angiograph(?:y|ie)?", r"angioplast(?:y|ie)?",
    r"\bpci\b", r"\bptca\b", r"\bstent\b", r"stent placement",
    r"balloon", r"dilation",
    r"percutaneous intervention"
]
kw_label = re.compile("|".join(label_patterns), flags=re.I)

# transfers.careunit 후보 (B)
unit_patterns = [
    r"cath\s*lab", r"cardiac\s*cath", r"interventional", r"cardio interventional",
    r"\bccl\b", r"ep/cath",
    r"icu.*cath|cath.*icu", r"\bpcu\b|\bccu\b",
    r"telemetry", r"monitoring\s*unit"
]
kw_unit = re.compile("|".join(unit_patterns), flags=re.I)

# ---- 시간대 안전 보정 ----
def to_utc_safe(s: pd.Series) -> pd.Series:
    """naive -> UTC localize, tz-aware -> UTC convert, null safe"""
    s = pd.to_datetime(s, errors="coerce", utc=False)
    try:
        if getattr(s.dt, "tz", None) is not None:
            return s.dt.tz_convert("UTC")
        else:
            return s.dt.tz_localize("UTC")
    except Exception:
        return pd.to_datetime(s.astype(str), errors="coerce", utc=True)

# ---- ICD prefilter ----
def get_likely_cath_hadm(conn, cohort_hadm_df):
    proc_icd = pd.read_sql("""
        SELECT subject_id, hadm_id, icd_version, icd_code
        FROM procedures_icd
        WHERE hadm_id IN (SELECT DISTINCT hadm_id FROM admissions)
    """, conn)

    proc_icd['subject_id'] = proc_icd['subject_id'].astype('Int64', errors='ignore')
    proc_icd['hadm_id']    = proc_icd['hadm_id'].astype('Int64', errors='ignore')
    proc_icd["code_clean"] = (proc_icd["icd_code"].astype(str)
                              .str.replace(".", "", regex=False)
                              .str.upper())

    # ICD-9: PCI(360x), Stent(00.66), Coronary angio(88.55)
    mask9 = (proc_icd["icd_version"]==9) & (
        proc_icd["code_clean"].str.startswith(("360")) |
        proc_icd["code_clean"].str.match(r"(0066|8855)", na=False)
    )
    # ICD-10-PCS: Coronary angio(B211/B212), PCI(0270~0273)
    mask10 = (proc_icd["icd_version"]==10) & (
        proc_icd["code_clean"].str.match(r"(B211|B212|0270|0271|0272|0273)", na=False)
    )

    # 옵션: CABG 포함
    if INCLUDE_CABG_AS_CATH:
        # ICD-9 CABG 36.1x, ICD-10-PCS 0210~0213 (보수적 범위)
        mask9  = mask9  | ((proc_icd["icd_version"]==9)  & proc_icd["code_clean"].str.match(r"361\d", na=False))
        mask10 = mask10 | ((proc_icd["icd_version"]==10) & proc_icd["code_clean"].str.match(r"021[0-3]", na=False))

    likely = proc_icd[mask9 | mask10][["subject_id","hadm_id"]].drop_duplicates()
    likely = likely.merge(cohort_hadm_df[["subject_id","hadm_id"]].drop_duplicates(),
                          on=["subject_id","hadm_id"], how="inner")

    del proc_icd; gc.collect()
    return likely

# ---- 이상치 컷 ----
def apply_sanity_window(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col not in df.columns or df.empty:
        return df
    lo = -float(ALLOW_NEG_MINUTES)
    hi = float(MAX_MINUTES_WINDOW)
    return df[(df[col].notna()) & (df[col] >= lo) & (df[col] <= hi)].copy()

print("\n=== Cell 5: Door-to-Cath 추출 (판단 기준 수정 v2.2) 시작 ===")
try:
    conn = get_db_connection(DB_PATH)

    # 0) ICD prefilter
    if USE_PROC_ICD_PREFILTER:
        likely_cath = get_likely_cath_hadm(conn, cohort2)
        hadms = sorted(likely_cath["hadm_id"].dropna().unique().tolist())
        print(f"procedures_icd 기반 cath-가능 hadm: {len(hadms)}")
    else:
        hadms = sorted(cohort2["hadm_id"].dropna().unique().tolist())

    # d_items & cohort core
    d_items = pd.read_sql("SELECT itemid, label FROM d_items", conn)
    d_items["label_lc"] = d_items["label"].astype(str).str.lower()

    cohort_core = cohort2[["subject_id","hadm_id","intime"]].drop_duplicates().copy()
    cohort_core["intime"] = to_utc_safe(cohort_core["intime"])

    # ---- A) procedureevents 기반 ----
    BATCH = 2000
    proc_parts = []
    for i in range(0, len(hadms), BATCH):
        batch = hadms[i:i+BATCH]
        placeholders = ",".join(["?"]*len(batch))
        pe = pd.read_sql(f"""
            SELECT subject_id, hadm_id, starttime, endtime, itemid
            FROM procedureevents
            WHERE hadm_id IN ({placeholders}) AND starttime IS NOT NULL
        """, conn, params=batch)
        if pe.empty:
            continue

        pe["starttime"] = to_utc_safe(pe["starttime"])
        pe["endtime"]   = to_utc_safe(pe["endtime"])

        pe = pe.merge(d_items[["itemid","label_lc"]], on="itemid", how="left")
        pe_cath = pe[pe["label_lc"].astype(str).str.contains(kw_label, na=False)].copy()
        if pe_cath.empty:
            continue

        # merge 후 intime 명시 리네임(intime_door) — KeyError 방지
        pe_cath = pe_cath.merge(cohort_core, on=["subject_id","hadm_id"], how="inner")
        if "intime_door" not in pe_cath.columns and "intime" in pe_cath.columns:
            pe_cath = pe_cath.rename(columns={"intime": "intime_door"})

        pe_cath["door_to_cath_min_A"] = (pe_cath["starttime"] - pe_cath["intime_door"]).dt.total_seconds()/60.0
        pe_cath = apply_sanity_window(pe_cath, "door_to_cath_min_A")

        gA = (pe_cath.sort_values("door_to_cath_min_A")
                      .groupby(["subject_id","hadm_id"], as_index=False)
                      .first()[["subject_id","hadm_id","door_to_cath_min_A"]])
        proc_parts.append(gA)

    cath_A = (pd.concat(proc_parts, ignore_index=True)
              if proc_parts else
              pd.DataFrame(columns=["subject_id","hadm_id","door_to_cath_min_A"]))

    # ---- B) transfers 기반 ----
    trf_parts = []
    for i in range(0, len(hadms), BATCH):
        batch = hadms[i:i+BATCH]
        placeholders = ",".join(["?"]*len(batch))
        tr = pd.read_sql(f"""
            SELECT subject_id, hadm_id, intime, outtime, careunit
            FROM transfers
            WHERE hadm_id IN ({placeholders}) AND intime IS NOT NULL
        """, conn, params=batch)
        if tr.empty:
            continue

        tr["intime"]  = to_utc_safe(tr["intime"])
        tr["outtime"] = to_utc_safe(tr["outtime"])
        tr["careunit_lc"] = tr["careunit"].astype(str).str.lower()

        tr_cath = tr[tr["careunit_lc"].str.contains(kw_unit, na=False)].copy()
        if tr_cath.empty:
            continue

        tr_cath = tr_cath.merge(cohort_core, on=["subject_id","hadm_id"], how="inner", suffixes=("","_door"))
        tr_cath["door_to_cath_min_B"] = (tr_cath["intime"] - tr_cath["intime_door"]).dt.total_seconds()/60.0
        tr_cath = apply_sanity_window(tr_cath, "door_to_cath_min_B")

        gB = (tr_cath.sort_values("door_to_cath_min_B")
                      .groupby(["subject_id","hadm_id"], as_index=False)
                      .first()[["subject_id","hadm_id","door_to_cath_min_B"]])
        trf_parts.append(gB)

    conn.close()

    cath_B = (pd.concat(trf_parts, ignore_index=True)
              if trf_parts else
              pd.DataFrame(columns=["subject_id","hadm_id","door_to_cath_min_B"]))

    # ---- A 우선, 없으면 B로 보완 + 출처 플래그 ----
    door_to_cath = (cohort2[["subject_id","hadm_id"]]
                    .drop_duplicates()
                    .merge(cath_A, on=["subject_id","hadm_id"], how="left")
                    .merge(cath_B, on=["subject_id","hadm_id"], how="left"))

    door_to_cath["door_to_cath_min"] = door_to_cath["door_to_cath_min_A"].combine_first(
        door_to_cath["door_to_cath_min_B"]
    )

    # dtype 충돌 방지: 문자열 컬럼은 string dtype + pd.NA로 생성
    condA = door_to_cath["door_to_cath_min_A"].notna()
    condB = door_to_cath["door_to_cath_min_B"].notna()

    door_to_cath["source"] = pd.Series(pd.NA, index=door_to_cath.index, dtype="string")
    door_to_cath.loc[condB, "source"] = "B_transfers"
    door_to_cath.loc[condA, "source"] = "A_procedureevents"

    door_to_cath["inferred"] = door_to_cath["source"].eq("B_transfers").fillna(False)

    # 최종 결과
    cath_df = door_to_cath.copy()

    # 리포트
    n_detect = cath_df['door_to_cath_min'].notna().sum()
    med_val  = cath_df.loc[cath_df['door_to_cath_min'].notna(), 'door_to_cath_min'].median()
    n_A      = cath_df['door_to_cath_min_A'].notna().sum()
    n_B      = cath_df['door_to_cath_min_B'].notna().sum()
    print("\n✅ Door-to-Cath 개선된 탐지 결과:")
    print(f"  • Door-to-Cath (A or B): {n_detect} 명")
    print(f"  • A(procedureevents) 단독 탐지: {n_A} 명")
    print(f"  • B(transfers) 단독/보완 탐지: {n_B} 명")
    print(f"  • Door-to-Cath (Median): {med_val:.1f} 분")
    print(f"  • Inferred(B) 비율: {cath_df['inferred'].sum()} / {n_detect} = {cath_df['inferred'].mean():.1%}")

except Exception as e:
    print(f"❌ ERROR in Cell 5 v2.2: {e}")
    raise



=== Cell 5: Door-to-Cath 추출 (판단 기준 수정 v2.2) 시작 ===
procedures_icd 기반 cath-가능 hadm: 464

✅ Door-to-Cath 개선된 탐지 결과:
  • Door-to-Cath (A or B): 276 명
  • A(procedureevents) 단독 탐지: 65 명
  • B(transfers) 단독/보완 탐지: 264 명
  • Door-to-Cath (Median): 176.5 분
  • Inferred(B) 비율: 211 / 276 = 45.5%


In [45]:
# === Cell 7: PCI 시술 여부 병합 및 최종 저장 (Door-to-Cath 포함) ===
print("\n=== Cell 7: PCI 시술 여부 병합 및 최종 저장 시작 ===")

try:
    conn = get_db_connection(DB_PATH)
    print("procedures_icd 테이블 로드 중...")

    # PCI 관련 ICD 코드
    pci_icd9 = ["0066", "3601", "3602", "3605", "3606", "3607", "3609"]
    pci_icd10_prefix = ["0270", "0271", "0272", "0273"]

    df_proc = pd.read_sql("""
        SELECT subject_id, hadm_id, icd_code, icd_version
        FROM procedures_icd
    """, conn)
    conn.close()

    # 전처리
    df_proc["icd_code"] = df_proc["icd_code"].astype(str).str.replace(".", "", regex=False).str.upper()
    df_proc["subject_id"] = df_proc["subject_id"].astype("int32")
    df_proc["hadm_id"] = df_proc["hadm_id"].astype("Int32")

    # PCI 필터
    mask9 = (df_proc["icd_version"]==9) & (df_proc["icd_code"].isin(pci_icd9))
    mask10 = (df_proc["icd_version"]==10) & (df_proc["icd_code"].str.startswith(tuple(pci_icd10_prefix)))
    pci_df = df_proc[mask9 | mask10][["subject_id","hadm_id"]].drop_duplicates()

    # PCI 여부 플래그 생성
    pci_df["PCI_flag"] = True

    print(f"PCI 시술 환자 수: {len(pci_df)}")

    # 코호트에 병합
    cohort_final_with_pci = cohort_final.merge(
        pci_df, on=["subject_id","hadm_id"], how="left"
    )
    cohort_final_with_pci["PCI_flag"] = cohort_final_with_pci["PCI_flag"].fillna(False)

    # 최종 저장
    SAVE_PATH_PCI = "/content/drive/MyDrive/final_cohort_with_timevars_pci.csv"
    cohort_final_with_pci.to_csv(SAVE_PATH_PCI, index=False, encoding="utf-8-sig")

    print(f"\n✅ PCI 병합 완료 및 최종 저장 완료")
    print(f"저장 경로: {SAVE_PATH_PCI}")
    print(f"최종 Shape: {cohort_final_with_pci.shape}")
    print(f"PCI=True 환자 수: {cohort_final_with_pci['PCI_flag'].sum()}")

except Exception as e:
    print(f"❌ ERROR in Cell 7: {e}")
    import traceback; traceback.print_exc()



=== Cell 7: PCI 시술 여부 병합 및 최종 저장 시작 ===
procedures_icd 테이블 로드 중...
PCI 시술 환자 수: 8075

✅ PCI 병합 완료 및 최종 저장 완료
저장 경로: /content/drive/MyDrive/final_cohort_with_timevars_pci.csv
최종 Shape: (465, 22)
PCI=True 환자 수: 465


In [42]:
import os, glob
import pandas as pd

# -------------------------------
# 0) 후보 디렉터리(있는 쪽 자동 사용)
# -------------------------------
CANDIDATE_DIRS = [
    "/content/drive/MyDrive", # Colab/Drive
    ".",                      # 현재 폴더
    "/mnt/data"               # 업로드 폴더
]

pattern = "final_cohort_with_timevars_pci*.csv"
found = []
for d in CANDIDATE_DIRS:
    if os.path.isdir(d):
        found.extend(glob.glob(os.path.join(d, pattern)))

if not found:
    raise FileNotFoundError("final_cohort_with_timevars_pci*.csv 파일을 어느 디렉터리에서도 찾지 못했습니다.")

# 파일 수정시간 기준 정렬(최신이 앞)
found = sorted(found, key=lambda p: os.path.getmtime(p), reverse=True)

# 최신(new)과 직전(old) 선택
path_new = found[0]
path_old = found[1] if len(found) > 1 else None

print("🔎 찾은 파일(최신→오래된 순):")
for p in found:
    print(" -", p)
print(f"\n🆕 new: {path_new}")
print(f"🗂️ old: {path_old}\n")

# -------------------------------
# 1) 로드
# -------------------------------
new = pd.read_csv(path_new)
old = pd.read_csv(path_old) if path_old else None

# -------------------------------
# 2) new에서 door_to_cath_min 복구(비어있는 경우)
#    - _x/_y가 남아있으면 합치고,
#    - 그래도 비어있으면 old에서 값만 가져와 새로 채움
# -------------------------------
def combine_xy(df, base_name):
    x, y = f"{base_name}_x", f"{base_name}_y"
    if x in df.columns or y in df.columns:
        df[base_name] = df.get(x).combine_first(df.get(y))
        df.drop(columns=[c for c in [x, y] if c in df.columns], inplace=True, errors="ignore")
    return df

# 2-1) _x/_y 정리
new = combine_xy(new, "door_to_cath_min")

# 2-2) 아직도 비어있으면 old에서 복구
needs_restore = ("door_to_cath_min" not in new.columns) or new["door_to_cath_min"].isna().all()
if needs_restore and old is not None:
    # old에서도 _x/_y 정리 후 가져오기
    old = combine_xy(old, "door_to_cath_min")
    if "door_to_cath_min" in old.columns:
        restore = old[["subject_id","hadm_id","door_to_cath_min"]].copy()
        # new에 동일 컬럼이 있으면 일단 삭제 후 merge
        new = new.drop(columns=["door_to_cath_min"], errors="ignore")
        new = new.merge(restore, on=["subject_id","hadm_id"], how="left")
        print("🧩 door_to_cath_min 값을 old에서 복구했습니다.")
    else:
        print("⚠️ old 파일에도 door_to_cath_min이 없어 복구할 수 없습니다.")

# -------------------------------
# 3) 반올림(+정수화) 처리
# -------------------------------
if "door_to_cath_min" in new.columns:
    # 숫자 변환 후 반올림
    new["door_to_cath_min"] = pd.to_numeric(new["door_to_cath_min"], errors="coerce")
    before_nonnull = new["door_to_cath_min"].notna().sum()
    new["door_to_cath_min"] = new["door_to_cath_min"].round(0).astype("Int64")
    after_nonnull = new["door_to_cath_min"].notna().sum()
    print(f"✅ 반올림/정수화 완료 (non-null: {before_nonnull} → {after_nonnull})")
else:
    print("⚠️ door_to_cath_min 컬럼이 최종적으로도 없습니다. (복구 실패)")

# -------------------------------
# 4) 최종 저장(덮어쓰기)
# -------------------------------
new.to_csv(path_new, index=False, encoding="utf-8-sig")
print(f"💾 최종 덮어쓰기 저장 완료: {path_new}")

# 간단 검증 출력
if "door_to_cath_min" in new.columns:
    print("미리보기:", new["door_to_cath_min"].head(10).tolist())
    print("non-null count:", int(new["door_to_cath_min"].notna().sum()))


🔎 찾은 파일(최신→오래된 순):
 - /content/drive/MyDrive/final_cohort_with_timevars_pci.csv

🆕 new: /content/drive/MyDrive/final_cohort_with_timevars_pci.csv
🗂️ old: None

✅ 반올림/정수화 완료 (non-null: 0 → 0)
💾 최종 덮어쓰기 저장 완료: /content/drive/MyDrive/final_cohort_with_timevars_pci.csv
미리보기: [<NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>]
non-null count: 0


In [46]:
import pandas as pd

# 파일 경로
file_path = "/content/drive/MyDrive/final_cohort_with_timevars_pci.csv"

# CSV 불러오기
df = pd.read_csv(file_path)

# 숫자 변환 (안 되는 값은 NaN)
for c in ["door_to_cath_min_y", "door_to_cath_min_x"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 유효 범위 기준 (−30분 ~ 4320분 = 72시간)
valid_y = df["door_to_cath_min_y"].between(-30, 4320, inclusive="both") if "door_to_cath_min_y" in df.columns else None
valid_x = df["door_to_cath_min_x"].between(-30, 4320, inclusive="both") if "door_to_cath_min_x" in df.columns else None

# Y 우선, 없을 때만 정상 범위 X로 대체
cand_y = df["door_to_cath_min_y"].where(valid_y) if valid_y is not None else pd.Series([pd.NA]*len(df))
cand_x = df["door_to_cath_min_x"].where(valid_x) if valid_x is not None else pd.Series([pd.NA]*len(df))

df["door_to_cath_min"] = cand_y.combine_first(cand_x)

# 반올림 + 정수형 변환
df["door_to_cath_min"] = pd.to_numeric(df["door_to_cath_min"], errors="coerce").round(0).astype("Int64")

# 🧹 불필요한 컬럼 제거
drop_cols = ["door_to_cath_min_x", "door_to_cath_min_y", "source", "inferred", "PCI_flag"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

# 저장 (덮어쓰기)
df.to_csv(file_path, index=False, encoding="utf-8-sig")

# 결과 확인
n_valid = df["door_to_cath_min"].notna().sum()
print(f"✅ 정리 완료: door_to_cath_min 유효값 {n_valid} / {len(df)}")
print(f"🧹 제거된 컬럼: {[c for c in drop_cols if c not in df.columns]}")
print(f"💾 저장 완료: {file_path}")



✅ 정리 완료: door_to_cath_min 유효값 277 / 465
🧹 제거된 컬럼: ['door_to_cath_min_x', 'door_to_cath_min_y', 'source', 'inferred', 'PCI_flag']
💾 저장 완료: /content/drive/MyDrive/final_cohort_with_timevars_pci.csv


In [55]:
import pandas as pd

# 1️⃣ 실제 파일 경로 (Colab Drive 기준)
src_path = "/content/drive/MyDrive/final_cohort_with_timevars_pci.csv"

# 2️⃣ 저장 파일명 (개선 버전)
save_path = "/content/drive/MyDrive/final_cohort_with_timevars_pci_refined.csv"

# 3️⃣ 데이터 불러오기
df = pd.read_csv(src_path)
print(f"📂 원본 불러오기 완료: {df.shape}")

# 4️⃣ door_to_cath_min 숫자 변환
df["door_to_cath_min"] = pd.to_numeric(df["door_to_cath_min"], errors="coerce")

# 5️⃣ 이상치 조건 정의
lower_bound, upper_bound = -30, 4320  # (-30분 ~ 72시간)

# 6️⃣ 이상치 → NaN 처리
mask_invalid = (df["door_to_cath_min"] < lower_bound) | (df["door_to_cath_min"] > upper_bound)
n_invalid = mask_invalid.sum()
df.loc[mask_invalid, "door_to_cath_min"] = pd.NA

# 7️⃣ 반올림 및 정수형 변환
df["door_to_cath_min"] = df["door_to_cath_min"].round(0).astype("Int64")

# 8️⃣ 저장
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"""
✅ 저장 완료: {save_path}
- 제거된 이상치(−30분 미만 or 72시간 초과): {n_invalid}건
- 유효값(non-null): {df['door_to_cath_min'].notna().sum()} / {len(df)}
""")


📂 원본 불러오기 완료: (465, 18)

✅ 저장 완료: /content/drive/MyDrive/final_cohort_with_timevars_pci_refined.csv
- 제거된 이상치(−30분 미만 or 72시간 초과): 0건
- 유효값(non-null): 277 / 465



In [60]:
import pandas as pd

# 1️⃣ 대상 파일 경로
path = "/content/drive/MyDrive/final_cohort_with_timevars_pci_refined.csv"

# 2️⃣ 남길 컬럼 (나머지는 자동 제거)
keep_cols = [
    "subject_id",
    "hadm_id",
    "intime",
    "outtime",
    "admittime",
    "lab_tat_min",
    "door_to_antithrombotic_min",
    "door_to_cath_min",
    "boarding_delay_min"
]

# 3️⃣ 파일 불러오기
df = pd.read_csv(path)

# 4️⃣ 지정된 컬럼만 남기기
df = df[[c for c in keep_cols if c in df.columns]]

# 5️⃣ 저장 (덮어쓰기)
df.to_csv(path, index=False, encoding="utf-8-sig")

print(f"""
✅ 불필요 컬럼 제거 완료
- 남은 컬럼 수: {len(df.columns)}
- 남은 컬럼 목록: {list(df.columns)}
💾 저장 완료: {path}
""")



✅ 불필요 컬럼 제거 완료
- 남은 컬럼 수: 9
- 남은 컬럼 목록: ['subject_id', 'hadm_id', 'intime', 'outtime', 'admittime', 'lab_tat_min', 'door_to_antithrombotic_min', 'door_to_cath_min', 'boarding_delay_min']
💾 저장 완료: /content/drive/MyDrive/final_cohort_with_timevars_pci_refined.csv



In [ ]:
!git config --global user.email "ji85673@gmail.com"
!git config --global user.name "ji85673"

%cd /content/drive/MyDrive
!git clone https://github.com/sosomeet/DA_STEMI.git
%cd DA_STEMI

!cp "/content/drive/MyDrive/timevars_analysis.ipynb" .
!cp "/content/drive/MyDrive/final_cohort_with_timevars_pci_refined (1).csv" ./final_cohort_with_timevars_pci_refined.csv

!git add .
!git commit -m "Door-to-Cath(PCI) 추출 및 시간 변수 정제 코드 추가"
!git push origin main
